<a href="https://colab.research.google.com/github/advantage-mercury/mercury-ds/blob/main/upgini_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I was finally able to access their free API by connecting to their website through my Mac Mini (I don't know why that worked), but after struggling with the code for a long time and
getting inconsistent messages, I pinged search.upgini.com, their repository for all their keyed data, and it came back 404. I will try again in the future. Their repo and
documentation have not been updated in a while as well.

In [1]:
%pip install -Uq upgini catboost
%pip install -Uq scikit-learn
%pip install -Uq xgboost

In [2]:
import pandas as pd
import socket
import requests
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType
from upgini import SearchKey
from upgini.ads import upload_user_ads

In [3]:
df = pd.read_csv('/content/sample_data/Sales_Product_Price_by_Store.csv')

In [4]:
df["Store"] = df["Store"].astype(str)
df["Product"] = df["Product"].astype(str)

# Convert date column to datetime pandas object
df["Date"] = pd.to_datetime(df["Date"])

df.sort_values("Date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,Store,Product,Date,Is_Holiday,Base Price,Price,Weekly_Units_Sold
0,1,1,2010-02-05,False,9.99,7.99,245
1,7,2,2010-02-05,False,5.99,5.99,141
2,3,3,2010-02-05,False,19.99,19.99,31
3,3,2,2010-02-05,False,5.99,5.99,117
4,8,3,2010-02-05,False,19.99,19.99,70


In [5]:
X = df[df["Date"] < "2011-01-01"]
y = df[df["Date"] >= "2012-10-26"]

In [6]:
from upgini import FeaturesEnricher, SearchKey

enricher = FeaturesEnricher(
 search_keys = {
   "Date":SearchKey.DATE,
    # "key_datetime": SearchKey.DATETIME,
    # "key_phone": SearchKey.PHONE,
    # "key_email": SearchKey.EMAIL,
    # "key_hashed_email": SearchKey.HEM,
    # "key_ip_address": SearchKey.IP
 },
#   keep_input = True,
#  api_key = "hSux-OfcWglRfQUetAPbfCFrqcqLxj4iAq31DgZfoLY"
)

In [7]:
train_features = df.drop(['Weekly_Units_Sold'], axis=1)
train_target = df.Weekly_Units_Sold
enriched_train_features = enricher.fit_transform(
    train_features,
    train_target,
    scoring = "mean_absolute_error")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION



Column name,Status,Errors
Date,All valid,-
target,All valid,-



Running search request, search_id=bed18b89-161e-43a9-a07a-f4fbf565b498
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


45 relevant feature(s) found with the search keys: ['Date']


f_events_date_year_cos7_9580852c,0.0193,100.0000,"-0.8977, 0.9088, -0.2297",Upgini,Calendar data,Free
f_weather_date_weather_umap_10_6b2de688,0.0146,100.0000,"5.1634, 5.496, 5.2188",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_4_0c0d6662,0.0143,100.0000,"3.8514, 3.8514, 4.3758",Upgini,Weather & climate normals data,Premium
f_events_date_year_sin3_e4c1ec0e,0.0138,100.0000,"-0.9997, -0.622, -0.9176",Upgini,Calendar data,Free
f_weather_date_weather_pca_1_9e4af931,0.0117,100.0000,"19.7702, -4.3547, -1.4661",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_11_c0a11c2b,0.0105,100.0000,"3.8956, 5.2168, 4.5078",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_1_68679173,0.0057,100.0000,"8.3491, 4.8604, 7.3761",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_6_8de4f955,0.0049,100.0000,"11.8544, 3.201, 6.8514",Upgini,Weather & climate normals data,Premium
f_events_date_year_cos1_9014a856,0.0047,100.0000,"-0.49, -0.1284, 0.5416",Upgini,Calendar data,Free
f_weather_date_weather_umap_21_1e3366b6,0.0046,100.0000,"4.962, 5.0477, 4.9931",Upgini,Weather & climate normals data,Premium
f_events_date_year_sin2_59955ffd,0.0038,100.0000,"-0.622, 0.4775, -0.4775",Upgini,Calendar data,Free


Upgini,Weather & climate normals data,0.0903,19
Upgini,Calendar data,0.0452,7
Upgini,World economic indicators,0.0182,13
Upgini,Markets data,0.0105,6


We detected 176 outliers in your sample.
Examples of outliers with maximum value of target:
165    2557
173    2524
43     2153
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Before dropping target outliers size: 3861
After dropping target outliers size: 3685
Calculating accuracy uplift after enrichment...


Train,3685,180.1180,39.3140,31.3891,7.9249


In [8]:
from upgini.metadata import CVType
enricher = FeaturesEnricher(
	search_keys={"Date": SearchKey.DATE},
	cv=CVType.time_series
)

In [9]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
from sklearn.model_selection import train_test_split

# Find all categorical features and replace NaNs with 'NA'
cat_col_enriched = [col for col in enriched_train_features.columns if enriched_train_features[col].dtype == "O"]
enriched_train_features.loc[:, cat_col_enriched] = enriched_train_features.loc[:, cat_col_enriched].fillna("NA")

cat_col_baseline = [col for col in train_features.columns if train_features[col].dtype == "O"]
train_features.loc[:, cat_col_baseline] = train_features.loc[:, cat_col_baseline].fillna("NA")

# Train and test split for correct model evaluation
X_train, X_test, y_train, y_test, X_train_baseline, X_test_baseline = train_test_split(
    enriched_train_features,
    train_target,
    train_features,
    test_size=0.2,
    shuffle=True,
    random_state=0)

# Task-optimized Catboost estimator
model = CatBoostRegressor(
    learning_rate=0.03,
    iterations=330,
    random_state=0,
    eval_metric="MAE",
    verbose=False,)

In [10]:
model.fit(X_train_baseline, y_train, cat_features=cat_col_baseline)
preds = model.predict(X_test_baseline)
eval_metric(y_test.values, preds, "MAE")

[34.63767164622421]

In [11]:
model.fit(X_train, y_train, cat_features=cat_col_enriched)
preds = model.predict(X_test)
eval_metric(y_test.values, preds, "MAE")

[26.26390673108738]